In [76]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [77]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [78]:
num_cars = 2 # Number of cars
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head(5))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: "None"


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [79]:
data = data.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)
data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.f"))

In [80]:
data.select('timestamp').dtypes

[Datetime(time_unit='ns', time_zone=None)]

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

### Ingenieria de variables

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [81]:
print(data.collect())

shape: (12_603, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[ns]               ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 26.122443 ┆ -116.580611 ┆ null          │
│ car_0  ┆ 2023-01-01 00:30:00        ┆ 26.122375 ┆ -116.578975 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:39:53.710416 ┆ 26.122375 ┆ -116.578975 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:40:52.213535 ┆ 26.122375 ┆ -116.578975 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 01:00:00        ┆ 26.11961  ┆ -116.58178  ┆ null          │
│ …      ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_1  ┆ 2023-01-07 23:57:43.425317 ┆ 21.756305 ┆ -89.552457  ┆ engine_check 

In [82]:
data = (data
    .sort(['car_id', 'timestamp'])
    .with_columns([
        pl.col('timestamp').diff().over('car_id').fill_null(pl.duration(nanoseconds=0)).alias('notification_time')
    ])
)

In [83]:
print(data.collect())

shape: (12_603, 6)
┌────────┬─────────────────────┬───────────┬─────────────┬───────────────┬───────────────────┐
│ car_id ┆ timestamp           ┆ latitude  ┆ longitude   ┆ notification  ┆ notification_time │
│ ---    ┆ ---                 ┆ ---       ┆ ---         ┆ ---           ┆ ---               │
│ str    ┆ datetime[ns]        ┆ f64       ┆ f64         ┆ str           ┆ duration[ns]      │
╞════════╪═════════════════════╪═══════════╪═════════════╪═══════════════╪═══════════════════╡
│ car_0  ┆ 2023-01-01 00:00:00 ┆ 26.122443 ┆ -116.580611 ┆ null          ┆ 0ns               │
│ car_0  ┆ 2023-01-01 00:30:00 ┆ 26.122375 ┆ -116.578975 ┆ low_fuel      ┆ 30m               │
│ car_0  ┆ 2023-01-01          ┆ 26.122375 ┆ -116.578975 ┆ tire_pressure ┆ 9m 53s 710416µs   │
│        ┆ 00:39:53.710416     ┆           ┆             ┆               ┆                   │
│ car_0  ┆ 2023-01-01          ┆ 26.122375 ┆ -116.578975 ┆ low_fuel      ┆ 58s 503119µs      │
│        ┆ 00:40:52.213535     

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [84]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # rodio de la tierra en km
    R = 6371.0

    # Convertir a radianes
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    # diferencias
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # formula de haversine
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

In [85]:
#calculamos la distancia entre dos puntos
data = data.with_columns(
    pl.when(pl.col("car_id") == pl.col("car_id").shift(1))
    .then(
        haversine(
            pl.col("latitude"),
            pl.col("longitude"),
            pl.col("latitude").shift(1),
            pl.col("longitude").shift(1)
        )
    )
    .otherwise(pl.lit(0))
    .alias("distance")
)

In [86]:
print(data.collect())

shape: (12_603, 7)
┌────────┬──────────────────┬───────────┬─────────────┬───────────────┬─────────────────┬──────────┐
│ car_id ┆ timestamp        ┆ latitude  ┆ longitude   ┆ notification  ┆ notification_ti ┆ distance │
│ ---    ┆ ---              ┆ ---       ┆ ---         ┆ ---           ┆ me              ┆ ---      │
│ str    ┆ datetime[ns]     ┆ f64       ┆ f64         ┆ str           ┆ ---             ┆ f64      │
│        ┆                  ┆           ┆             ┆               ┆ duration[ns]    ┆          │
╞════════╪══════════════════╪═══════════╪═════════════╪═══════════════╪═════════════════╪══════════╡
│ car_0  ┆ 2023-01-01       ┆ 26.122443 ┆ -116.580611 ┆ null          ┆ 0ns             ┆ 0.0      │
│        ┆ 00:00:00         ┆           ┆             ┆               ┆                 ┆          │
│ car_0  ┆ 2023-01-01       ┆ 26.122375 ┆ -116.578975 ┆ low_fuel      ┆ 30m             ┆ 0.163503 │
│        ┆ 00:30:00         ┆           ┆             ┆               ┆ 

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [87]:
intervalo = "5m"

In [88]:
# Agrupar por 'car_id' y por intervalos de tiempo en 'timestamp'
info_notificacion = data.group_by_dynamic(
    group_by="car_id", index_column="timestamp", every=intervalo, period=intervalo, closed="both", include_boundaries=True
).agg([
    pl.col("notification_time").mean().alias("media"),
    pl.col("notification_time").median().alias("mediana"),
    pl.col("notification_time").var().alias("varianza"),
    pl.col("notification_time").max().alias("max"),
    pl.col("notification_time").min().alias("min")
])

print(info_notificacion.select('car_id', 'timestamp', 'media', 'mediana', 'varianza', 'max', 'min').collect())

shape: (2_765, 7)
┌────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ car_id ┆ timestamp    ┆ media        ┆ mediana      ┆ varianza     ┆ max          ┆ min          │
│ ---    ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│ str    ┆ datetime[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] │
╞════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ car_0  ┆ 2022-12-31   ┆ 0ns          ┆ 0ns          ┆ null         ┆ 0ns          ┆ 0ns          │
│        ┆ 23:55:00     ┆              ┆              ┆              ┆              ┆              │
│ car_0  ┆ 2023-01-01   ┆ 0ns          ┆ 0ns          ┆ null         ┆ 0ns          ┆ 0ns          │
│        ┆ 00:00:00     ┆              ┆              ┆              ┆              ┆              │
│ car_0  ┆ 2023-01-01   ┆ 30m          ┆ 30m          ┆ null         ┆ 30

In [89]:
info_distancia = data.group_by_dynamic(
group_by="car_id", index_column="timestamp", every=intervalo, period=intervalo, closed="both", include_boundaries=True
).agg([
    pl.col("distance").mean().alias("media"),
    pl.col("distance").median().alias("mediana"),
    pl.col("distance").var().alias("varianza"),
    pl.col("distance").max().alias("max"),
    pl.col("distance").min().alias("min")
])

print(info_distancia.select('car_id', 'timestamp', 'media', 'mediana', 'varianza', 'max', 'min').collect())

shape: (2_765, 7)
┌────────┬─────────────────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ car_id ┆ timestamp           ┆ media    ┆ mediana  ┆ varianza ┆ max      ┆ min      │
│ ---    ┆ ---                 ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str    ┆ datetime[ns]        ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞════════╪═════════════════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ car_0  ┆ 2022-12-31 23:55:00 ┆ 0.0      ┆ 0.0      ┆ null     ┆ 0.0      ┆ 0.0      │
│ car_0  ┆ 2023-01-01 00:00:00 ┆ 0.0      ┆ 0.0      ┆ null     ┆ 0.0      ┆ 0.0      │
│ car_0  ┆ 2023-01-01 00:25:00 ┆ 0.163503 ┆ 0.163503 ┆ null     ┆ 0.163503 ┆ 0.163503 │
│ car_0  ┆ 2023-01-01 00:30:00 ┆ 0.163503 ┆ 0.163503 ┆ null     ┆ 0.163503 ┆ 0.163503 │
│ car_0  ┆ 2023-01-01 00:35:00 ┆ 0.0      ┆ 0.0      ┆ null     ┆ 0.0      ┆ 0.0      │
│ …      ┆ …                   ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
│ car_1  ┆ 202